In [4]:
import numpy as np
import pandas as pd

# ----------------------------
# 1) Robertson ODE and Jacobian
# ----------------------------
def f_robertson(t, y):
    """Right-hand side f(t,y) for Robertson."""
    y1, y2, y3 = y
    return np.array([
        -0.04*y1 + 1e4*y2*y3,
        0.04*y1 - 1e4*y2*y3 - 3e7*y2*y2,
        3e7*y2*y2
    ], dtype=float)

def J_robertson(t, y):
    """Jacobian df/dy for Robertson."""
    y1, y2, y3 = y
    return np.array([
        [-0.04,       1e4*y3,        1e4*y2],
        [ 0.04, -1e4*y3 - 6e7*y2,   -1e4*y2],
        [ 0.0,        6e7*y2,         0.0]
    ], dtype=float)

# ----------------------------
# 2) Work counter (fair cost metric)
# ----------------------------
class WorkCounter:
    """Count model work: RHS evals, Jacobian evals, Newton iterations."""
    def __init__(self):
        self.nfev = 0
        self.nJ = 0
        self.nnewton = 0

# ----------------------------
# 3) Newton solver for Backward Euler step
#    y_{n+1} = y_n + h f(t_{n+1}, y_{n+1})
# ----------------------------
def backward_euler_step(t, y, h, counter: WorkCounter,
                        newton_max=12, newton_tol=1e-12):
    """
    One Backward Euler step from (t,y) to (t+h, y_new).
    Uses Newton iterations on G(y_new)= y_new - y - h f(t+h, y_new).
    """
    t_new = t + h
    yk = y.copy()  # Newton initial guess: previous value

    for _ in range(newton_max):
        counter.nnewton += 1

        fk = f_robertson(t_new, yk); counter.nfev += 1
        Jk = J_robertson(t_new, yk); counter.nJ += 1

        # Residual G(yk)
        G = yk - y - h * fk

        # Newton matrix: I - h J
        A = np.eye(3) - h * Jk

        # Solve for update delta
        delta = np.linalg.solve(A, -G)
        yk = yk + delta

        # Convergence check
        if np.linalg.norm(delta, ord=np.inf) < newton_tol:
            return yk, True  # success

    return yk, False  # Newton failed to converge within budget

# ----------------------------
# 4) Build Gauss-Legendre nodes on (0,1) and integration matrix Q
#    Q[i,j] ~= ∫_0^{c_i} l_j(s) ds,  i=0..m, j=1..m
#    with c_0=0, c_1..c_m = Gauss nodes in (0,1)
# ----------------------------
def build_Q_gauss(m):
    """
    Returns:
      c: length m array of Gauss-Legendre nodes mapped to (0,1)
      Q: (m+1) x m matrix, where Q[i,j] = ∫_0^{c_i} l_j(s) ds (approx exact for poly)
    Notes:
      We build Lagrange basis via polynomial fitting (stable enough for m<=16-ish).
    """
    r, _w = np.polynomial.legendre.leggauss(m)  # nodes in (-1,1)
    c = 0.5*(r + 1.0)                           # map to (0,1)

    Q = np.zeros((m+1, m), dtype=float)         # row 0 corresponds to c0=0

    for j in range(m):
        # Construct Lagrange basis l_j by fitting degree m-1 polynomial
        yvals = np.zeros(m); yvals[j] = 1.0
        coeff = np.polyfit(c, yvals, deg=m-1)    # polynomial coeffs, high->low
        coeff_int = np.polyint(coeff)           # integral polynomial coeffs

        for i in range(1, m+1):
            ci = c[i-1]
            Q[i, j] = np.polyval(coeff_int, ci) - np.polyval(coeff_int, 0.0)

    return c, Q

# ----------------------------
# 5) EuImp step on [t, t+h] with (m,J)
#    - nodes: c0=0, c1..cm (Gauss)
#    - initial guess: substepped backward Euler march
#    - perform J SDC sweeps driven by backward-Euler-type implicit solves
#    - return y(t+h) via polynomial extrapolation to s=1
# ----------------------------
def euimp_step(t, y0, h, m, J, counter: WorkCounter,
               newton_max=10, newton_tol=1e-12):
    """
    One implicit SDC (EuImp) step for Robertson.
    Returns (y_end, success).
    """
    c, Q = build_Q_gauss(m)
    t_nodes = t + h * c  # node times (excluding left endpoint)

    # u_prev[i] stores approximation at c_i, with i=0..m, c0=0
    u_prev = np.zeros((m+1, 3), dtype=float)
    u_prev[0] = y0.copy()

    # --- Initial nodal guess: backward Euler march across sub-intervals
    for i in range(m):
        c_left = 0.0 if i == 0 else c[i-1]
        c_right = c[i]
        hi = h * (c_right - c_left)
        t_left = t + h * c_left

        y_guess, ok = backward_euler_step(t_left, u_prev[i], hi, counter,
                                          newton_max=newton_max, newton_tol=newton_tol)
        if not ok:
            return y_guess, False
        u_prev[i+1] = y_guess

    # f_prev at nodes (only need nodes 1..m)
    f_prev = np.zeros((m+1, 3), dtype=float)
    for i in range(1, m+1):
        f_prev[i] = f_robertson(t_nodes[i-1], u_prev[i]); counter.nfev += 1

    # --- SDC sweeps
    for _k in range(J):
        u_new = np.zeros_like(u_prev)
        u_new[0] = y0.copy()

        for i in range(m):
            # Substep length
            c_left = 0.0 if i == 0 else c[i-1]
            c_right = c[i]
            hi = h * (c_right - c_left)

            ti1 = t_nodes[i]  # time at node i+1

            # Quadrature increment using previous sweep values:
            # dq = h * Σ_j ( (Q[i+1,j]-Q[i,j]) * f_prev[j+1] )
            dQ = Q[i+1, :] - Q[i, :]
            dq = h * (dQ @ f_prev[1:])  # f_prev[1:] corresponds to nodes 1..m

            # Implicit Euler SDC node equation rearranged into Newton form:
            # u - hi*f(ti1,u) = u_new[i] + dq - hi*f_prev[i+1]
            rhs = u_new[i] + dq - hi * f_prev[i+1]

            # Newton solve for node value u_new[i+1]
            uk = u_prev[i+1].copy()  # good initial guess: previous sweep node
            ok_node = False
            for _it in range(newton_max):
                counter.nnewton += 1
                fk = f_robertson(ti1, uk); counter.nfev += 1
                Jk = J_robertson(ti1, uk); counter.nJ += 1

                G = uk - hi*fk - rhs
                A = np.eye(3) - hi*Jk
                delta = np.linalg.solve(A, -G)
                uk = uk + delta

                if np.linalg.norm(delta, ord=np.inf) < newton_tol:
                    ok_node = True
                    break

            if not ok_node:
                return uk, False

            u_new[i+1] = uk

        # Update f_prev for next sweep
        for i in range(1, m+1):
            f_prev[i] = f_robertson(t_nodes[i-1], u_new[i]); counter.nfev += 1

        u_prev = u_new

    # --- Extrapolate nodal polynomial to s=1 (right endpoint)
    s_nodes = c
    y_end = np.zeros(3, dtype=float)
    for d in range(3):
        coeff = np.polyfit(s_nodes, u_prev[1:, d], deg=m-1)
        y_end[d] = np.polyval(coeff, 1.0)

    return y_end, True

# ----------------------------
# 6) Estimate mu(m,J) for EuImp on scalar y' = λy with huge negative λ
#    (cache once for EuComb)
# ----------------------------
def estimate_mu(m, J, lam=-1e12):
    """
    Approximate μ(m,J) = lim_{|λ|→∞} amplification of EuImp(m,J),
    by running scalar EuImp on y' = λy with very large negative λ on [0,1].
    """
    r, _w = np.polynomial.legendre.leggauss(m)
    c = 0.5*(r + 1.0)

    # Build Q on (0,1)
    Q = np.zeros((m+1, m), dtype=float)
    for j in range(m):
        yvals = np.zeros(m); yvals[j] = 1.0
        coeff = np.polyfit(c, yvals, deg=m-1)
        coeff_int = np.polyint(coeff)
        for i in range(1, m+1):
            ci = c[i-1]
            Q[i, j] = np.polyval(coeff_int, ci) - np.polyval(coeff_int, 0.0)

    # One step, h=1, y0=1
    y0 = 1.0
    u_prev = np.zeros(m+1, dtype=float)
    u_prev[0] = y0

    # Initial BE march: y_{i+1} = y_i / (1 - hi*λ)
    for i in range(m):
        hi = c[i] - (0.0 if i == 0 else c[i-1])
        u_prev[i+1] = u_prev[i] / (1.0 - hi*lam)

    f_prev = lam * u_prev

    # SDC sweeps (scalar, linear solve closed-form)
    for _k in range(J):
        u_new = np.zeros_like(u_prev)
        u_new[0] = y0
        for i in range(m):
            hi = c[i] - (0.0 if i == 0 else c[i-1])
            dQ = Q[i+1, :] - Q[i, :]
            dq = (dQ @ f_prev[1:])  # h=1
            rhs = u_new[i] + dq - hi*f_prev[i+1]
            u_new[i+1] = rhs / (1.0 - hi*lam)
        u_prev = u_new
        f_prev = lam * u_prev

    # Extrapolate to s=1
    coeff = np.polyfit(c, u_prev[1:], deg=m-1)
    y1 = np.polyval(coeff, 1.0)
    return float(y1)

# ----------------------------
# 7) EuComb step: combine two EuImp schemes with cached mus
# ----------------------------
def eucomb_step(t, y, h, params1, params2, mu1, mu2, counter: WorkCounter):
    """
    EuComb = (mu1*EuImp2 - mu2*EuImp1)/(mu1 - mu2)
    """
    (m1, J1) = params1
    (m2, J2) = params2

    y1, ok1 = euimp_step(t, y, h, m1, J1, counter)
    if not ok1:
        return y1, False
    y2, ok2 = euimp_step(t, y, h, m2, J2, counter)
    if not ok2:
        return y2, False

    return (mu1*y2 - mu2*y1) / (mu1 - mu2), True

# ----------------------------
# 8) Adaptive driver with step-doubling error estimate
# ----------------------------
def adaptive_solve(method_name, stepper, y0, T, tol,
                   h0=1e-6, hmin=1e-16, hmax=0.1):
    """
    Generic adaptive integrator using:
      - step-doubling error estimate
      - accept/reject with halving
      - double step size after 2 consecutive accepts
    Returns:
      yT, stats(dict), history(list of (t,h))
    """
    t = 0.0
    y = y0.copy()
    h = h0

    counter = WorkCounter()
    n_accept = 0
    n_reject = 0
    consec_accept = 0
    hist = []

    while t < T:
        h = min(h, T - t)
        if h < hmin:
            raise RuntimeError(f"{method_name}: step size underflow (h={h}).")

        # One step of size h
        y_big, ok_big = stepper(t, y, h, counter)
        if not ok_big:
            # Treat Newton failure as reject
            n_reject += 1
            consec_accept = 0
            h *= 0.5
            continue

        # Two half steps for error estimate
        y_half, ok1 = stepper(t, y, 0.5*h, counter)
        if not ok1:
            n_reject += 1
            consec_accept = 0
            h *= 0.5
            continue

        y_small, ok2 = stepper(t + 0.5*h, y_half, 0.5*h, counter)
        if not ok2:
            n_reject += 1
            consec_accept = 0
            h *= 0.5
            continue

        # Error indicator (∞-norm)
        err = np.linalg.norm(y_small - y_big, ord=np.inf)

        if err <= tol:
            # Accept
            t += h
            y = y_small  # use better (two-half-step) solution
            n_accept += 1
            consec_accept += 1
            hist.append((t, h))

            # Increase step size cautiously (paper-like rule)
            if consec_accept >= 2:
                h = min(2.0*h, hmax)
                consec_accept = 0
        else:
            # Reject
            n_reject += 1
            consec_accept = 0
            h *= 0.5

    stats = dict(
        method=method_name,
        accept_steps=n_accept,
        reject_steps=n_reject,
        nfev=counter.nfev,
        nJ=counter.nJ,
        nnewton=counter.nnewton
    )
    return y, stats, hist

# ----------------------------
# 9) Wrap steppers for BE / EuImp / EuComb
# ----------------------------
def make_BE_stepper():
    def stepper(t, y, h, counter):
        ynew, ok = backward_euler_step(t, y, h, counter)
        return ynew, ok
    return stepper

def make_EuImp_stepper(m=8, J=4):
    def stepper(t, y, h, counter):
        return euimp_step(t, y, h, m, J, counter)
    return stepper

def make_EuComb_stepper(params1, params2, mu1, mu2):
    def stepper(t, y, h, counter):
        return eucomb_step(t, y, h, params1, params2, mu1, mu2, counter)
    return stepper

# ----------------------------
# 10) Run the experiment
# ----------------------------
if __name__ == "__main__":
    T = 1.0
    tol = 1e-10
    y0 = np.array([1.0, 0.0, 0.0], dtype=float)

    # Cache μ values for EuComb
    mu_84 = estimate_mu(8, 4)
    mu_124 = estimate_mu(12, 4)
    if abs(mu_84 - mu_124) < 1e-14:
        raise RuntimeError("mu values too close; EuComb denominator unstable.")

    print("Cached mu:")
    print("  mu(8,4)  =", mu_84)
    print("  mu(12,4) =", mu_124)

    # Build steppers
    BE_stepper = make_BE_stepper()
    EuImp_stepper = make_EuImp_stepper(m=8, J=4)
    EuComb_stepper = make_EuComb_stepper((8,4), (12,4), mu_84, mu_124)

    # --- Reference solution (self-contained; no black-box solver)
    # We use EuComb but force a small max step to get a "tight" reference.
    def ref_stepper(t, y, h, counter):
        return eucomb_step(t, y, h, (8,4), (12,4), mu_84, mu_124, counter)

    y_ref, _ref_stats, _ = adaptive_solve(
        "Reference(EuComb-tight)",
        ref_stepper,
        y0, T,
        tol=1e-10,
        h0=1e-7, hmax=1e-4
    )

    # --- Run methods
    results = []
    yT_BE, stats_BE, _ = adaptive_solve("BackwardEuler", BE_stepper, y0, T, tol, h0=1e-6)
    yT_EI, stats_EI, _ = adaptive_solve("EuImp(8,4)", EuImp_stepper, y0, T, tol, h0=1e-6)
    yT_EC, stats_EC, _ = adaptive_solve("EuComb((8,4),(12,4))", EuComb_stepper, y0, T, tol, h0=1e-6)

    # Compute final-time errors vs reference
    for yT, st in [(yT_BE, stats_BE), (yT_EI, stats_EI), (yT_EC, stats_EC)]:
        st["err_T_inf"] = float(np.linalg.norm(yT - y_ref, ord=np.inf))
        results.append(st)

    df = pd.DataFrame(results)
    # Order columns nicely
    df = df[["method", "accept_steps", "reject_steps", "nfev", "nJ", "nnewton", "err_T_inf"]]
    print("\n=== Efficiency comparison (T=1, tol=1e-10) ===")
    print(df.to_string(index=False))


Cached mu:
  mu(8,4)  = -4.0634316331426007e-11
  mu(12,4) = -5.5918292592529734e-11

=== Efficiency comparison (T=1, tol=1e-10) ===
              method  accept_steps  reject_steps  nfev    nJ  nnewton    err_T_inf
       BackwardEuler          8547          4265 77194 77194    77194 3.112710e-07
          EuImp(8,4)            42             0 13389  8349     8349 8.870327e-11
EuComb((8,4),(12,4))            42             0 33098 20498    20498 9.502388e-11
